# 演習問題2
## Cntextial Inquiry法にWord2Vecを用いる

社会人講座第3回で作成したシナリオから分散表現を獲得し，スケジュール管理における不便さやストレスを探り，ITツールを活用してより効率的かつ正確に予定を管理する方法を見つける．

In [ ]:
import pandas as pd
import numpy as np
import MeCab
import ipadic
import re
from gensim.models import Word2Vec
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
from matplotlib import rcParams

#rcParamsに文字化けしないようにフォントの設定を行う
rcParams['font.family'] = 'sans-serif'
rcParams['font.sans-serif'] = ['Hiragino Maru Gothic Pro', 'Yu Gothic', 'Meirio', 'Takao', 'IPAexGothic', 'IPAPGothic', 'VL PGothic', 'Noto Sans CJK JP']


In [ ]:
# 形態素解析器の初期化
tagger = MeCab.Tagger(ipadic.MECAB_ARGS)
tagger.parse("")  # MeCabの初期化（空文字列を解析することで初期化）

### ストップワードの改良

In [ ]:
# 追加のストップワードを設定
additionel_stopwords = ["a", "いる", "か月", "する", "ユーザ"]

# ストップワードの読み込み
with open("stop_words.txt", encoding="utf-8") as f:
    stop_words = set(line.strip().lower() for line in f if line.strip())

# ストップワードを追加
stop_words.update(map(str.lower, additionel_stopwords))

### トークナイズにおける問題点

辞書によってはトークナイズの際におかしな分け方をしてしまう<br>
例：スマホ⇒[スマ, ホ]

In [ ]:
parsed = tagger.parse("メモを取る際、スマホを使う")
print(parsed)

カタカナ語が連続する場合，連結するようにトークナイズする

In [ ]:
# 英数字・記号などの正規化関数
def clean_token(surface):
    surface = surface.lower()
    surface = re.sub(r'[0-9０-９一二三四五六七八九十百千万億兆]+', '', surface)
    surface = re.sub(r'[^\wぁ-んァ-ン一-龥ー]+', '', surface)
    return surface

# トークナイズ＋カタカナ語の連結
def tokenize_clean(text):
    node = tagger.parseToNode(text)
    tokens = []
    buffer = ""  # カタカナ連続語用バッファ

    while node:
        surface = node.surface
        features = node.feature.split(',')
        pos = features[0]
        base = features[6] if len(features) > 6 and features[6] != '*' else surface
        norm = clean_token(base)

        # カタカナだけで構成されているならバッファに追加
        if re.fullmatch(r'[ァ-ンー]+', surface):
            buffer += surface
        else:
            # カタカナバッファがあればまず吐き出す
            if buffer:
                if buffer.lower() not in stop_words:
                    tokens.append(buffer.lower())
                buffer = ""
            # 通常の単語処理
            if pos in ["名詞", "動詞", "形容詞"] and norm and norm not in stop_words:
                tokens.append(norm)
        node = node.next

    # 最後にバッファを処理
    if buffer and buffer.lower() not in stop_words:
        tokens.append(buffer.lower())

    return tokens


In [ ]:
# スマホが[スマ, ホ]に分かれなくなる
print(tokenize_clean("メモを取る際、スマホを使う"))

### コーパスを読み込む
今回は，コーパスに皆様が作成されたシナリオを使用しています．<br>
分析の邪魔にならないように"・"はすべて削除しています．<br>
シナリオに書かれていた名前の部分はすべて"ユーザ"に書き換えています．

In [ ]:
# コーパスの読み込み
df = pd.read_csv("./csv/scenario_data.csv")

# トークン化
sentences = df["text"].apply(tokenize_clean)

### 同じ意味を持つ単語を統合
"pc"と"パソコン"など同じ意味を持つ単語を1語に統合することで，単語空間の過剰な分散を防げる

In [ ]:
# 意味の同じ単語を1語にまとめるための変換辞書
replace_dict = {
    "pc": "パソコン" # "pc"をすべて"パソコン"に置き換える
}

# 統合関数により単語の並びを変えることなく目的語を変換出来る
def batch_replace(sentences, replace_dict):
    return [
        [replace_dict.get(token, token) for token in sentence]
        for sentence in sentences
    ]

# 変換辞書を適用
sentences = batch_replace(sentences, replace_dict)

### Word2Vecの作成

In [ ]:
w2v_model = Word2Vec(
    sentences=sentences, 
    vector_size=100, 
    window=5, 
    min_count=2, 
    sg=1,
    seed=42
    )

In [ ]:
print(f"モデルの語彙数：{len(w2v_model.wv)}")

In [ ]:
# テキストを空白区切りにして渡す
joined_texts = [" ".join(tokens) for tokens in sentences]
vectorizer = TfidfVectorizer()
tfidf_matrix = vectorizer.fit_transform(joined_texts)
feature_names = vectorizer.get_feature_names_out()


In [ ]:
# すべてのシナリオのTF-IDF上位単語
for j in range(len(sentences)):
    row = tfidf_matrix[j].toarray().flatten()
    top_indices = row.argsort()[::-1][:5]
    top_words = [feature_names[i] for i in top_indices if feature_names[i] in w2v_model.wv]

    print(f"シナリオ{j}番目のTF-IDF上位語:", top_words)


In [ ]:
# 任意のシナリオのTF-IDF上位単語
scenarioID = 0
row = tfidf_matrix[scenarioID].toarray().flatten()
top_indices = row.argsort()[::-1][:5]
top_words = [feature_names[i] for i in top_indices if feature_names[i] in w2v_model.wv]

print(f"シナリオ{scenarioID}番目のTF-IDF上位語:", top_words)

# 平均ベクトルを求める
word_vecs = [w2v_model.wv[word] for word in top_words]
avg_vec = np.mean(word_vecs, axis=0)

# 周辺語を表示
similar_words = w2v_model.wv.similar_by_vector(avg_vec, topn=10)
print(f"シナリオ{scenarioID}番目の周辺語:")
similar_words

### 意味マップの画像化
任意の単語とその単語の類似度上位20単語からなる空間をPCAで2次元に圧縮し画像化する

In [ ]:
def plot_word_map_pca(center_word, model, topn=20):
    if center_word not in model.wv:
        print(f"{center_word} は語彙にありません。")
        return

    # 類似語取得
    similar_words = model.wv.most_similar(center_word, topn=topn)
    words = [center_word] + [w for w, _ in similar_words]
    vecs = np.array([model.wv[w] for w in words])

    # PCAで2次元に圧縮
    pca = PCA(n_components=2, random_state=42)
    vecs_2d = pca.fit_transform(vecs)

    # プロット
    plt.figure(figsize=(10, 8))
    for i, word in enumerate(words):
        x, y = vecs_2d[i]
        plt.scatter(x, y, color='red' if i == 0 else 'blue')
        plt.text(x, y, word, fontsize=12)
    plt.title(f"PCA: '{center_word}' を中心とした意味マップ")
    plt.grid(True)
    plt.show()


In [ ]:
target_word = "スマホ"

In [ ]:
print(w2v_model.wv.most_similar(target_word, topn=20))
plot_word_map_pca(target_word, w2v_model)


### 考察問題1 スマホを中心とした意味マップから本コーパスにおける"スマホ"はどのような使われ方をしているか考察してください

回答欄

## 課題 自分で前処理を設計して得られた結果から考察してください
唯一の答えはないので自由に考察してください．

### 操作1 ストップワードの改良
`stop_words.txt`を直接書き換えても良い

In [ ]:
# 追加のストップワードを設定
additionel_stopwords = []

# ストップワードの読み込み
with open("stop_words.txt", encoding="utf-8") as f:
    stop_words = set(line.strip().lower() for line in f if line.strip())

# ストップワードを追加
stop_words.update(map(str.lower, additionel_stopwords))

### 操作2 同じ意味を持つ単語の統合
"バスケ", "サッカー", "野球"をすべて"スポーツ"に書き換えるという使い方も可能

In [ ]:
# 意味の同じ単語を1語にまとめるための変換辞書
replace_dict = {
    
}

# 統合関数により単語の並びを変えることなく目的語を変換出来る
def batch_replace(sentences, replace_dict):
    return [
        [replace_dict.get(token, token) for token in sentence]
        for sentence in sentences
    ]


# トークン化
sentences = df["text"].apply(tokenize_clean)
# 変換辞書を適用
sentences = batch_replace(sentences, replace_dict)

### 操作3 Word2Vecの作成
パラメータを自分で設定してください

In [ ]:
model = Word2Vec(
    sentences=sentences,
    )

In [ ]:
print(f"モデルの語彙数：{len(model.wv)}")

In [ ]:
# 以下のコードで語彙に含まれるすべての単語を確認できる
# model.wv.index_to_key[:]

### 操作4 今日学んだ内容を振り返り，分析してください
セルはいくらでも追加してください．<br>
右上の`+`を押せばセルが追加されます．

### 考察問題2 以下の内容で考察してください
社会人講座第3回で作成したシナリオから分散表現を獲得し，スケジュール管理における不便さやストレスを探り，ITツールを活用してより効率的かつ正確に予定を管理する方法を見つける．

回答欄